In [8]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import pandas as pd
import time

In [9]:
geolocator = Nominatim(user_agent="CorrelateTemperatureToProvidence")

In [10]:
def geocode_reverse(location):
    try:
        # Add a throttle to account for geopy's timeout and limiter
        time.sleep(5)
        return geolocator.reverse(location)
    except GeocoderTimedOut:
        # Add a throttle to account for geopy's timeout and limiter
        time.sleep(5)
        return geocode_reverse(location)

In [11]:
def find_avg_temp_per_providence(dataset, output_path):
        providence_avg_temp = {}
        
        latitudes = dataset.get("Latitudes")
        longitudes = dataset.get("Longitudes")
        temperatures = dataset.get("Temperature")
        
        for lat, long, temp in zip(latitudes, longitudes, temperatures):
            location = geocode_reverse(str(lat) + "," + str(long))
            providence = location.raw["address"]["state"]
            
            if providence in providence_avg_temp:
                providence_avg_temp[providence].append(temp)
            else:
                providence_avg_temp[providence] = [temp]
        
        for key in providence_avg_temp.keys():
            temp_list = providence_avg_temp[key]
            
            total_temp_per_providence = 0
            avg_temp = 0
            for temp in temp_list:
                total_temp_per_providence += temp
            
            avg_temp = total_temp_per_providence/len(temp_list)
            providence_avg_temp[key] = avg_temp
        
        finalized_providence_temps = {}
        
        #Iterate in for loop to gurantee ordering
        finalized_providence_temps["Providences"] = []
        finalized_providence_temps["Temperatures"] = []
        
        for providence in providence_avg_temp:
            finalized_providence_temps["Providences"].append(providence)
            finalized_providence_temps["Temperatures"].append(providence_avg_temp[providence])
        
        data_frame = pd.DataFrame(finalized_providence_temps, columns=["Providences", "Temperatures"])
        data_frame.to_csv(output_path)       